In [60]:
from dotenv import load_dotenv
load_dotenv()

True

In [61]:
from dataclasses import dataclass

@dataclass
class Emailparams:
    email:str="shubhamsarkarthe1@gmail.com"
    password:str="sstr#1$"

In [62]:
from langchain.tools import tool,ToolRuntime
from langchain.agents import create_agent
from langchain.messages import HumanMessage,ToolMessage
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command

@tool
def read_inbox():
    """Reads the email content"""
    content=" We would like to talk about a new investment oppurtunity. Lets meet at the Taj. Are you up?"
    return content

@tool
def respond(to,subject,body):
    """Responds to sent email"""
    return f"email sent to {to} with subject {subject} and body {body}"

@tool
def authenticate(email,password,runtime:ToolRuntime):
    """Authenticte user with given email and password"""
    if email==runtime.context.email and password ==runtime.context.password:
        return Command(update={
            "authenticated":True,
            "messages":[ToolMessage("Sussessfully Authenticated",
                                    tool_call_id=runtime.tool_call_id)]
        })
    else:
        return Command(update={
            "authentication":False,
            "messages":[ToolMessage(
                "Authentication failed",
                tool_call_id=runtime.tool_call_id
            )]
        })

In [63]:
from langchain.agents import AgentState

class AuthenticatedState(AgentState):
    authenticated:bool=False

In [64]:
from langchain.agents.middleware import wrap_model_call,ModelRequest,ModelResponse,HumanInTheLoopMiddleware
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request:ModelRequest,handler:Callable[[ModelRequest],ModelResponse]):
    """Allow read inbox and send email tools only if user provides correct email and password"""
    authenticated=request.state.get("authenticated")
    

    if authenticated:
        tools=[respond,read_inbox] ## if email and password match user has access
    else:
        tools=[authenticate] # if email and password do not match user does not have access
      
    request = request.override(tools=tools) 
    return handler(request)

In [65]:
from langchain.agents.middleware import dynamic_prompt

authenticated_prompt = "You are a helpful assistant that can check the inbox and send emails."
unauthenticated_prompt = "You are a helpful assistant that can authenticate users."

@dynamic_prompt
def dynamic_prompt(request):
    """Generate system prompt based on authentication status"""
    authenticated=request.state.get("authenticated")

    if authenticated:
        return authenticated_prompt
    else:
        return unauthenticated_prompt

In [66]:
agent=create_agent(model="gpt-5-nano",
                   tools=[respond,read_inbox,authenticate],
                   context_schema=Emailparams,
                   state_schema=AuthenticatedState,
                   checkpointer=InMemorySaver(),
                   middleware=[dynamic_tool_call,dynamic_prompt,
                               HumanInTheLoopMiddleware(
                                   interrupt_on={
                                       "authenticate":False,
                                       "read_inbox":False,
                                        "respond":True
                                   }
                               )],
                   )

In [67]:
from langchain.messages import HumanMessage

config={"configurable":{"thread_id":"1"}}
response = agent.invoke(
    {"messages": [HumanMessage(content="Check my inbox")]},
    context=Emailparams
    ,config=config
)

print(response["messages"][-1].content)

I can help with that. I need to sign you in first.

Please provide:
- Email
- Password

Note: I’ll use these only to authenticate you for this request and won’t store them beyond this session. If you’re not comfortable sharing here, we can’t proceed with checking your inbox.


In [68]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="Please read my email and send a response")]},
    context=Emailparams,config=config
)

print(response["messages"][-1].content)

I can help draft and send a reply, but I’ll need to sign in to access your inbox first.

Two options:
1) Sign in to your account
- Please provide:
  - Email
  - Password
I’ll use these only for this session to read the inbox and send a reply on your behalf.

2) Draft without sharing credentials
- Paste the email content you want me to respond to (or summarize key points).
- Tell me the tone you want (formal, friendly, concise, urgent) and any points you want included.
- Let me know who to send it to, and any deadline or attachments to mention.

If you choose option 2, I can draft a response now and you can copy-paste it into your email client. If you choose option 1, I’ll proceed to log in and handle reading and replying.


In [69]:
from langchain.messages import HumanMessage

config={"configurable":{"thread_id":"1"}}
response = agent.invoke(
    {"messages": [HumanMessage(content="email:shubhamsarkarthe1@gmail.com,password:sstr#1$")]},
    context=Emailparams
    ,config=config
)

print(response["messages"][-1].content)

c:\Users\SHUBHAM SARKAR\Desktop\Agentic\lca-lc-foundations\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=<class '__main__.Emailparams'>, input_type=type])
  return self.__pydantic_serializer__.to_python(


I read your inbox. The message is about a new investment opportunity and proposes meeting at the Taj.

Here’s a ready-to-send reply. If you confirm the recipient email (the sender’s address) or tell me to reply to the same sender, I can send it right away.

Draft reply
Subject: Re: Investment opportunity

Body:
Hi there,

Thanks for reaching out about the investment opportunity. I’m interested in learning more and would be glad to meet at the Taj. Could you share a proposed date and time, as well as a brief agenda or details you’d like me to review beforehand? I’m available this week on Tuesday afternoon or Thursday afternoon, but I’m flexible—please let me know what works for you.

Best regards,
Shubham

What would you like to do next?
- Tell me the recipient’s email address (the From field of the incoming message) and I’ll send this for you.
- Or tell me to adjust the tone, add a phone number, or specify preferred times, and I’ll tailor the draft.


In [72]:
response = agent.invoke(
    {"messages": [HumanMessage(content="email of recipient:largeventures@gmail.com . send an email to this address confirming my avaliability in Sunday ")]},
    context=Emailparams
    ,config=config
)

print(response["messages"][-1].content)

In [73]:
from pprint import pprint
from langgraph.types import Command
response=agent.invoke(
    Command(
        resume={"decisions":[{"type":"approve"}]}
    ),
    config=config# same threadid
)
pprint(response)

{'authenticated': True,
 'messages': [HumanMessage(content='Check my inbox', additional_kwargs={}, response_metadata={}, id='5962a3e6-a29a-46bd-8681-960a651b167f'),
              AIMessage(content='I can help with that. I need to sign you in first.\n\nPlease provide:\n- Email\n- Password\n\nNote: I’ll use these only to authenticate you for this request and won’t store them beyond this session. If you’re not comfortable sharing here, we can’t proceed with checking your inbox.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 583, 'prompt_tokens': 146, 'total_tokens': 729, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsW8bgw0qjPsc4Vl6BARI0PtLz3jx', 'service_tier': 'default',

In [74]:
print(response["messages"][-1].content)

Email sent.

Details:
- To: largeventures@gmail.com
- Subject: Re: Investment opportunity
- Body sent:
Hi there,

Thanks for reaching out about the investment opportunity. I’m available this Sunday and would be glad to meet. If convenient, I can meet at the Taj as previously discussed. Please let me know a time that works for you, or suggest a slot on Sunday (e.g., 1-3 pm or 4-6 pm). I’ll adjust as needed.

Best regards,
Shubham

Would you like me to:
- Add a specific preferred time (e.g., 2:00-3:00 pm) or propose a precise slot?
- Include a phone number for easier coordination?
- Send a follow-up if you don’t hear back by a certain date?
